In [ ]:
import requests as req
import os, json, time, re
from bs4 import BeautifulSoup as bs
from urllib import parse
from fake_useragent import UserAgent
import ssl
# 針對SSL出現過期或其他故障修正
ssl._create_default_https_context = ssl._create_unverified_context
# 隨機取得 User-Agent
ua = UserAgent(use_external_data=True)

# 放置古騰堡小說metadata的資訊
listData = []

# 古騰堡小說的網址
url = 'https://www.gutenberg.org/browse/languages/zh'

# 設定標頭
headers = {
    'user-agent': ua.random
}

# 沒有放置txt檔的資料夾，就建立起來
folderPath = '古騰堡小說'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
# 取得小說的主要連結
def getMainLinks():
    # 走訪首頁
    res = req.get(url, headers = headers)
    soup = bs(res.text, "lxml")
    
    # 取得主要連結，透過li元素來選取a的超連結
    a_elms = soup.select('li.pgdbetext > a')
    
    # 整理主要連結資訊
    for a in a_elms:
        listData.append({
            "title": a.get_text(),
            "link": f'https://www.gutenberg.org{a["href"]}.html.noimages',
        })
# 建立古騰堡小說 json 檔
def saveJson():
    with open(f"{folderPath}/gutenberg.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False) )
        
# 將古騰堡小說所有章回的內容，各自寫到 txt 與 json 中
def writeTxt():
    # 稍候建立 train.json 前的程式變數
    listContent = []

    # 開啟 古騰堡小說 metadata 的 json 檔
    with open(f"{folderPath}/gutenberg.json", "r", encoding="utf-8") as file:
        strJson = file.read()

    # 走訪所有小說文字內容
    listResult = json.loads(strJson)
    for i in range(350):
        res = req.get(listResult[i]['link'], headers = headers)
        soup = bs(res.text, "lxml")
        #s = soup.select('p')
        strContent = []
        match = re.search(r"^[a-zA-Z]+",listResult[i]['title'])
        if match == None:
            for s in soup.select('p'):
                strContent.append(s.get_text())
                strContent2 = "".join(strContent)
                strContent2 = re.sub(r"[a-zA-Z\W\d]+" ,"",strContent2)
                
        # 決定 txt 的檔案名稱
                fileName = f"{listResult[i]['title']}.txt"
                fileName = re.sub(r" |\r|\n|　|\s", '', fileName)
            
            
        # 將小說內容存到 txt 中
            with open(f"{folderPath}/{fileName}", "w", encoding="utf-8") as file:
                file.write(strContent2)        
    

In [ ]:
# 主程式
if __name__=="__main__":
    getMainLinks()
    saveJson()
    writeTxt()